# This note for FID Score Calculation of our CGAN Model

Import all the required libraries

In [0]:
import numpy
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10
from tensorflow.keras.models import load_model
import os
from numpy.random import randint
from matplotlib import pyplot

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("/content/drive/My Drive/deeplearning/")
!ls

This model can be used to predict the feature vector for one or more images. Our images are likely to not have the required shape. We will use the scikit-image library to resize the NumPy array of pixel values to the required size. The scale images() function below implements this.

In [0]:
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)

Once resized, the image pixel values will also need to be scaled to meet the expectations for inputs to the inception model. This can be achieved by calling the preprocess input() function. We can update our calculate fid() function defined in the theory to take the loaded inception model and two NumPy arrays of image data as arguments, instead of activations. The function will then calculate the activations before calculating the FID score as before. The updated version of the calculate_fid() function is listed below.

In [0]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [0]:
real_data_file = "img_align_celeba_attractive_face.npz"
images1 = numpy.load(real_data_file)["arr_0"]
images1 = images1[:10000]
#images1 = (images1 - 127.5) / 127.5
#images1 = (images1 + 1) / 2.0
print(images1.shape)
#print(images1[0,:,:])
#print('Loaded', images1.shape, images2.shape)

(10000, 64, 64, 3)


In [0]:
model = load_model("results/models/generator_model_200.h5")
CGAN_data_file = "results/latent_points/latent_points_10000.npz"
latent_points = numpy.load(CGAN_data_file)["arr_0"]
print(latent_points.shape)
# generate images
labels = randint(0, 4, 10000)
print(labels)
images2 = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
#images2 = (images2 + 1) / 2.0
# scale from [-1,1] to [0,255]
images2 = (images2 * 127.5) + 127.5
print(images2.shape)
#print(images2[0,:,:])

(10000, 100)
[0 0 0 ... 2 1 3]
(10000, 64, 64, 3)


Then, we can load the Inception v3 model in Keras directly. This will prepare a version of the inception model for classifying images as one of 1,000 known classes. We can remove the output (the top) of the model via the include top=False argument. Painfully, this also removes the global average pooling layer that we require, but we can add it back via specifying the pooling=‘avg’ argument. When the output layer of the model is removed, we must specify the shape of the input images, which is 299 × 299 × 3 pixels, e.g. the input shape=(299,299,3) argument.

In [0]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
print('Preprocessed', images1.shape, images2.shape)
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

87916544/87910968 [==============================] - 1s 0us/step
Loaded (10000, 64, 64, 3) (10000, 64, 64, 3)
Scaled (10000, 299, 299, 3) (10000, 299, 299, 3)
Preprocessed (10000, 299, 299, 3) (10000, 299, 299, 3)
FID: 21.361
